# Thesis Experiment: t5 Model
## Michael LeVine, April 19, 2024
## Hyperparameter testing: train epochs 8

The purpose of this notebook is to test the summarization capabilities of the t5 mode after fine-tuning it for 8 training epochsl.

Attribution: This approach is based on a training course of Janana Ravi, Certified Google Cloud Architect and Data Engineer - from the LinkedIn Learning course: AI Text Summarization with Hugging Face, released 10/30/2023.

## Overview: Using a Transformer Model from Hugging Face: t5

### The t5 model
The pre-trained model that we will use is the "t5-small" model from Hugging Face, which can be found here: https://huggingface.co/google-t5/t5-small

The "model card," which describes the model, notes that it is a "text-to-text" transformer model.  The model card notes that t5 base is the model checkpoint with 60 million parameters

## Verifying the Compute Environment

### Graphics Processing Unit (GPU)
Running inference on transformer models can be done without a GPU.  However, for training, a GPU is recommended.  The following block of code shows checks whether a GPU is available for use in a PyTorch environment. 
.

In [1]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
print("using", device, "device")

using cuda device


## Installing and importing required libraries and dependencies

In [2]:
#command line pip install the necessary required libraries and dependencies
#the transformers library allows us to access the pre-trained t5-small model
#the datasets library provides access to the Hugging Face datasets
#the evaluate model enables us to evaluate the summarizations the model produces
#the rouge_score is a standard evaluation metric used in text summarization tasks
#the accelerate function allows for distributed training on GPUs
#pip install transformers datasets evaluate rouge_score accelerate

### Import the transformers library

In [3]:
#this code imports needed libraries

import transformers
import datasets
import evaluate
import rouge_score
import accelerate

print(transformers.__version__) # verifies the transformers version

4.32.1


## Importing, Reducing, and Exploring the dataset

The experiment will use the CNN/Daily Mail dataset.  Two datasets will be created:
* Training Dataset
* Holdout Dataset (for running inference on the model)


### Instantiating a training dataset

In [4]:
#loading the dataset which was previously saved
from datasets import load_dataset
cnn_news_summary_ds = load_dataset("arrow", data_files={'train': 'data/cnn_news_summary_ds/train/data-00000-of-00001.arrow', 'test': 'data/cnn_news_summary_ds/test/data-00000-of-00001.arrow'})


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
cnn_news_summary_ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 2296
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 575
    })
})

As per above output, now the dataset is broken down into two components:
* a `train` (training) dataset of 2296 articles
* a `test` dataset of 575



### Instantiating a holdout dataset (200 records)

The holdout dataset is used to run inference on both the "off-the-shelf" model and the fine-tuned model.  The purpose of having a holdout set is so the model is running inference on a different dataset from what it was trained on in order to test its performance.

In [6]:
#load holdout set for inference from a local csv (to ensure same order)
cnn_holdout_ds = load_dataset ("csv", data_files='data/cnn_holdout_ds.csv', split = "train[0:200]")
cnn_holdout_ds

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/envs/gpu/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 200
})

### Exploring the Dataset

In [7]:
#dataset shape
cnn_news_summary_ds.shape

{'train': (2296, 3), 'test': (575, 3)}

The above output shows the cnn_dailymail `train` set is 2296 rows x 3 columns.  The `test` set is 575 rows x 3 columns.

In [8]:
#dataset object type
type(cnn_news_summary_ds)

datasets.dataset_dict.DatasetDict

The above output shows the cnn_dailymail dataset is of type Dataset, within the datasets library.

In [9]:
#dataset structure
cnn_news_summary_ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 2296
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 575
    })
})

The Dataset has three features: 
* `article`: The full text of the news article
*  `highlights`: the target summary, also known as the reference summary
*  `id`: the unique id for each article/highlights pair


In [10]:
#looking at the features of the dataset
cnn_news_summary_ds['train'].features

{'article': Value(dtype='string', id=None),
 'highlights': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None)}

In [11]:
#examining the first record of the dataset.  
cnn_news_summary_ds['train'][0]

{'article': '(CNN) -- After almost 10 months, the FBI has zeroed in on a suspect in the case of missing Florida pilot Robert Wiles, who may have been kidnapped for ransom. Missing Florida pilot Robert Wiles is thought to have been kidnapped for ransom. "We\'re close to solving the case," said FBI special agent David Couvertier. He would not elaborate. Agents also would not identify the suspect, and they said the person is not in custody. Investigators would only reveal that the "key suspect" is in Florida, either in Orlando, Lakeland or Melbourne. "They\'re holding that back in hopes of getting additional information," said Couvertier. The FBI says it\'s also looking at several persons of interest in those same three Florida cities. Wiles, 27, was last seen in the family\'s aircraft maintenance business, National Flight Services, at Lakeland Linder Regional Airport on April 1, 2008. The day Wiles disappeared, he left behind his bags, his computer, and even his car. His father says the 

## Preprocessing and Cleaning the Data

In [12]:
#defining a text cleaning function.  This function iterates over the 'article'
# and 'highlights' section and replaces various text strings (like
#backslashes, new lines, etc.) with the empty string

def clean_txt(example):
  for txt in ['article', 'highlights']:
    example[txt] = example[txt].lower() #convert text to lowercase
    example[txt] = example[txt].replace('\\','')
    example[txt] = example[txt].replace('/','')
    example[txt] = example[txt].replace('\n','')
    example[txt] = example[txt].replace('``','')
    example[txt] = example[txt].replace('"','')
    example[txt] = example[txt].replace('--','')
  return example

### Mapping the text cleaning function to the training dataset and the holdout dataset

In [13]:
#hugging face datasets allow the .map operation to
#apply a function to all records in a dataset, and then will
#update the dataset.  In efect, the .map() method
# maps the `clean_txt` function to
# all the records in the `cnn_news_summary_ds dataset.
#The result will be that the training and
#test data will now be cleaned
cleaned_cnn_news_summary_ds = cnn_news_summary_ds.map(clean_txt)

cleaned_cnn_news_summary_ds

Map:   0%|          | 0/2296 [00:00<?, ? examples/s]

Map:   0%|          | 0/575 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 2296
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 575
    })
})

In [14]:
#verifying that we have a clean training dataset by comparing a record from the original
#dataset to the cleaned dataset
print('\n\n==== Original dataset ====\n\n')

print (cnn_news_summary_ds['train']['article'][0])

print('\n\n==== Cleaned dataset ====\n\n')

cleaned_cnn_news_summary_ds['train']['article'][0]



==== Original dataset ====


(CNN) -- After almost 10 months, the FBI has zeroed in on a suspect in the case of missing Florida pilot Robert Wiles, who may have been kidnapped for ransom. Missing Florida pilot Robert Wiles is thought to have been kidnapped for ransom. "We're close to solving the case," said FBI special agent David Couvertier. He would not elaborate. Agents also would not identify the suspect, and they said the person is not in custody. Investigators would only reveal that the "key suspect" is in Florida, either in Orlando, Lakeland or Melbourne. "They're holding that back in hopes of getting additional information," said Couvertier. The FBI says it's also looking at several persons of interest in those same three Florida cities. Wiles, 27, was last seen in the family's aircraft maintenance business, National Flight Services, at Lakeland Linder Regional Airport on April 1, 2008. The day Wiles disappeared, he left behind his bags, his computer, and even his car. His fa

"(cnn)  after almost 10 months, the fbi has zeroed in on a suspect in the case of missing florida pilot robert wiles, who may have been kidnapped for ransom. missing florida pilot robert wiles is thought to have been kidnapped for ransom. we're close to solving the case, said fbi special agent david couvertier. he would not elaborate. agents also would not identify the suspect, and they said the person is not in custody. investigators would only reveal that the key suspect is in florida, either in orlando, lakeland or melbourne. they're holding that back in hopes of getting additional information, said couvertier. the fbi says it's also looking at several persons of interest in those same three florida cities. wiles, 27, was last seen in the family's aircraft maintenance business, national flight services, at lakeland linder regional airport on april 1, 2008. the day wiles disappeared, he left behind his bags, his computer, and even his car. his father says the next day, wiles was supp

In [15]:
cleaned_cnn_holdout_ds = cnn_holdout_ds.map(clean_txt)

cleaned_cnn_holdout_ds

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 200
})

In [16]:
#verifying that we have a clean holdout dataset by comparing a record from the original
#holdout dataset to the cleaned holdout dataset
print('\n\n==== Original holdout dataset ====\n\n')

print (cnn_holdout_ds['article'][0])

print('\n\n==== Cleaned holdout dataset ====\n\n')

cleaned_cnn_holdout_ds['article'][0]



==== Original holdout dataset ====


Manchester United have fallen off their perch. And they’re dropping like a stone towards mediocrity. That is the undeniable fact that has been hammered home relentlessly during the past six months. Whether we are talking about the events of Wednesday night at Olympiacos or before the startled eyes of the faithful at Old Trafford, the evidence is there for all to see. Can't stop the slump: David Moyes cannot believe it as he watches Manchester United lose at Olympiacos . Down and almost out: Robin van Persie lies on the floor during a defeat which sees United's Champions League campaign hanging by a thread . Disbelief: Wayne Rooney cries out in vain during another shambolic United display . Abject: The frustration shows on the Man United players' faces on taking the restart after conceding to Olympiacos . Coming to get you: Liverpool are looking to take United's place in the top four . Now is it time for Man United to sack Moyes? Out of the title r

"manchester united have fallen off their perch. and they’re dropping like a stone towards mediocrity. that is the undeniable fact that has been hammered home relentlessly during the past six months. whether we are talking about the events of wednesday night at olympiacos or before the startled eyes of the faithful at old trafford, the evidence is there for all to see. can't stop the slump: david moyes cannot believe it as he watches manchester united lose at olympiacos . down and almost out: robin van persie lies on the floor during a defeat which sees united's champions league campaign hanging by a thread . disbelief: wayne rooney cries out in vain during another shambolic united display . abject: the frustration shows on the man united players' faces on taking the restart after conceding to olympiacos . coming to get you: liverpool are looking to take united's place in the top four . now is it time for man united to sack moyes? out of the title race, out of the fa . cup, out of the l

In [17]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["highlights"], max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

## Instantiating an "off-the-shelf" t5 model and tokenizer

In [18]:
from transformers import pipeline, T5ForConditionalGeneration, TrainingArguments, Trainer, \
                         DataCollatorForSeq2Seq, T5Tokenizer
import pandas as pd
from datasets import Dataset
import random

In [19]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [20]:
#looking at model architecture
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

## Preprocessing Datasets

In [21]:
#define a preprocessing function

prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["highlights"], max_length=256, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

### Preprocessing the training dataset

In [22]:
tokenized_cnn_training_ds = cleaned_cnn_news_summary_ds.map(preprocess_function, batched=True)
tokenized_cnn_training_ds

Map:   0%|          | 0/2296 [00:00<?, ? examples/s]

Map:   0%|          | 0/575 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2296
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 575
    })
})

### Preprocessing the holdout dataset

In [23]:
tokenized_cnn_holdout_ds = cleaned_cnn_holdout_ds.map(preprocess_function, batched=True)
tokenized_cnn_holdout_ds

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 200
})

## Instantiating a Data Collator

In [24]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Importing ROUGE Metric
*  Rouge is a standard evaluation metric used in text summarization tasks
*  ROUGE provides an *objective metric* to compare model-produced summary with the dataset's reference summary.  

### Importing the evaluate library

The 'evaluate' library from Hugging Face allows us to evaluate ML models.  The 'evaluate' library provides access to dozens of evaluation metrics across many ML domains (including NLP, computer vision, etc.).

In [25]:
import evaluate

rouge = evaluate.load("rouge")
rouge

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

## Fine-tuning the t5-small model

In [26]:
#defining a custom compute metrics function
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [27]:
training_args = Seq2SeqTrainingArguments(
    output_dir="t5_small_results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16, 
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=8,#changed train_epochs from 4 to 8
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    load_best_model_at_end=True, #even if we overtrain model by accident, we will still 
    #load the checkpoint that had lowest evaluation loss

    #evaluation_strategy can be steps or epochs - correlates to how often we stop training and evaluate our model
    eval_steps=50,
    save_strategy='epoch' #save the model after every epoch
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_cnn_training_ds["train"],
    eval_dataset=tokenized_cnn_training_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
    
)
trainer.evaluate() #adding a max_new_tokens paramater)

/opt/conda/envs/gpu/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/envs/gpu/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 2.0412049293518066,
 'eval_rouge1': 0.2202,
 'eval_rouge2': 0.0915,
 'eval_rougeL': 0.1821,
 'eval_rougeLsum': 0.1816,
 'eval_gen_len': 19.0,
 'eval_runtime': 33.0248,
 'eval_samples_per_second': 17.411,
 'eval_steps_per_second': 0.545}

In [28]:
training_args = Seq2SeqTrainingArguments(
    output_dir="t5_small_results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=8,#changed train_epochs from 4 to 8
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    load_best_model_at_end=True, #even if we overtrain model by accident, we will still 
    #load the checkpoint that had lowest evaluation loss

    #evaluation_strategy can be steps or epochs - correlates to how often we stop training and evaluate our model
    eval_steps=50,
    save_strategy='epoch' #save the model after every epoch
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_cnn_training_ds["train"],
    eval_dataset=tokenized_cnn_training_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train() #adding a max_new_tokens paramater)

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.782240,0.219600,0.091100,0.182400,0.182000,19.000000
2,No log,1.676981,0.216900,0.089400,0.180800,0.180600,19.000000
3,No log,1.653257,0.216800,0.091500,0.182300,0.182200,19.000000
4,No log,1.640851,0.215800,0.089700,0.180800,0.180600,19.000000
5,No log,1.634546,0.216100,0.089700,0.180700,0.180600,19.000000
6,No log,1.630149,0.217400,0.090100,0.182000,0.181700,19.000000
7,1.889600,1.627690,0.217100,0.090300,0.181500,0.181400,19.000000
8,1.889600,1.627046,0.217200,0.090400,0.181600,0.181400,19.000000


/opt/conda/envs/gpu/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/envs/gpu/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/envs/gpu/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/envs/gpu/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input 

TrainOutput(global_step=576, training_loss=1.8796703815460205, metrics={'train_runtime': 1836.9148, 'train_samples_per_second': 9.999, 'train_steps_per_second': 0.314, 'total_flos': 4971916418875392.0, 'train_loss': 1.8796703815460205, 'epoch': 8.0})

In [29]:
trainer.save_model()

## Instantiating the fine_tuned model and running inference with that model

In [30]:
fine_tuned_checkpoint = "t5_small_results"

In [31]:
#instantiating the fine-tuned model
fine_tuned_model = T5ForConditionalGeneration.from_pretrained(fine_tuned_checkpoint)

In [32]:
fine_tuned_model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [33]:
#instantiating the fine-tuned tokeinizer
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_checkpoint)

### Running inference on holdout set with "fine-tuned" t5-small model

In [34]:
#instantiating a summarizer pipeline with the fine-tuned t5-small model
fine_tuned_summarizer = pipeline ('summarization', model=fine_tuned_checkpoint)

In [35]:
#creating 'holdout_article_texts' variable to hold articles from the test set
holdout_article_texts = tokenized_cnn_holdout_ds["article"]

#creating 'holdout_article_summaries' variable to hold summaries from the test set
holdout_article_summaries = tokenized_cnn_holdout_ds["highlights"]


In [36]:
#running inference on holdout set with fine-tuned
from tqdm import tqdm

#instantiating an empty list named 'holdout summaries'
holdout_fine_tuned_summaries = []

prefix ='summarize: '

for i, text in enumerate(tqdm(holdout_article_texts[:500])):

    candidate = fine_tuned_summarizer(prefix + text)

    holdout_fine_tuned_summaries.append(candidate[0]["summary_text"])

100%|█████████████████████████████████████████| 200/200 [04:44<00:00,  1.42s/it]


## Evaluating the fine-tuned t5 performance

In [37]:
#aggregated results of inference on holdout set using fine-tune dmodel
result_fine_tuned_agg = rouge.compute(predictions = holdout_fine_tuned_summaries,
                           references = holdout_article_summaries[:],
                           use_stemmer=True)

result_fine_tuned_agg

{'rouge1': 0.36781610903619916,
 'rouge2': 0.17243510950973093,
 'rougeL': 0.2652166112167161,
 'rougeLsum': 0.2657194953331504}

In [38]:
#unaggregated results of inference on holdout set using fine-tune dmodel
result_fine_tuned_unagg = rouge.compute(predictions = holdout_fine_tuned_summaries,
                           references = holdout_article_summaries[:],
                           use_stemmer=True,
                            use_aggregator=False)
                                      

result_fine_tuned_unagg

{'rouge1': [0.25,
  0.36363636363636365,
  0.38235294117647056,
  0.3741935483870968,
  0.3508771929824561,
  0.25925925925925924,
  0.44186046511627913,
  0.3661971830985915,
  0.48387096774193544,
  0.29333333333333333,
  0.41975308641975306,
  0.46031746031746035,
  0.35,
  0.35185185185185186,
  0.6436781609195402,
  0.28125,
  0.17857142857142858,
  0.2947368421052632,
  0.6233766233766234,
  0.4571428571428572,
  0.43956043956043955,
  0.6923076923076923,
  0.32432432432432434,
  0.59375,
  0.16161616161616163,
  0.21428571428571427,
  0.4186046511627907,
  0.48739495798319327,
  0.31249999999999994,
  0.6567164179104478,
  0.4235294117647059,
  0.4,
  0.37623762376237624,
  0.2456140350877193,
  0.39473684210526316,
  0.3565891472868217,
  0.45528455284552843,
  0.34693877551020413,
  0.41791044776119407,
  0.4090909090909091,
  0.5565217391304349,
  0.5858585858585857,
  0.346938775510204,
  0.32098765432098764,
  0.3442622950819672,
  0.3934426229508197,
  0.42352941176470593,

In [64]:
cleaned_cnn_holdout_df.to_csv('data/data_frame_after_t5.csv')

In [65]:
test_df = pd.read_csv('data/data_frame_after_t5.csv')
test_df

,Unnamed: 0,article,highlights,id,t5_summaries,t5_fine_tuned_summaries
0,0,manchester united have fallen off their perch....,manchester united were beaten 2-0 in their cha...,5b3a626078390cb0e05327b4019753fd11cb8cea,manchester united lost 1-0 to olympiacos in th...,manchester united lost 1-0 to olympiacos in th...
1,1,a mother whose russian husband snatched their ...,"rachael neustadt's sons - daniel, eight and jo...",59d478d4a4299e2192997e56a9db9003fa2bac2d,"rachael neustadt's sons daniel, eight, and jon...","rachael neustadt's sons daniel, eight, and jon..."
2,2,claim: supporters of mayor lutfur rahman alleg...,islamic voters allegedly told to be 'good musl...,ec961b7d0912e7753dffe4360b77481eba96f2e1,supporters of mayor lutfur rahman allegedly ha...,supporters of mayor lutfur rahman allegedly ha...
3,3,the 15-year-old cousin of a palestinian boy wh...,"mohammed abu khder, 16, abducted and burned to...",092d90d61eb105b3955820cc4894ac2c4995ad1b,"mohammed abu khder, 16, was abducted from his ...","mohammed abu khder, 16, was burned to death in..."
4,4,it may have made its way up the pole to become...,spearmint rhino records £2.1m loss in 2011 .lo...,d0d59018cdf48aaeb6e1838c0323f8555e800765,spearmint rhino has recorded a loss of £2.1mil...,spearmint rhino has recorded a loss of £2.1m i...
...,...,...,...,...,...,...
195,195,reality tv show the block has been accused of ...,'the block' caught out faking a visit from the...,985b1bf7fc710e4ffdd9dd02e72d889a7997e89d,reality tv show the block has been accused of ...,reality tv show the block has been accused of ...
196,196,the average cost of raising a child to seconda...,average cost of raising a child from birth up ...,e466296e19d7a14cf4916d70a2cbc296e4659c99,average cost of raising a child to secondary s...,average cost of raising a child to secondary s...
197,197,thai police investigating the murder of two br...,"pornprasit sukdam claims he was offered £13,30...",a07624a84fe59a3321e83f153d6fd615207a8545,"pornprasit sukdam claims he was offered 700,00...","pornprasit sukdam claims he was offered 700,00..."
198,198,from clumpy flat shoes that seem to shorten a ...,clumpy flat shoes that seem to shorten a woman...,e16474b52bbf45f49434fc4a0b1d68e2d3fba3c3,kim carillo says she feels surprisingly sexy i...,"kim carillo, who usually favours a more alluri..."
